<a href="https://colab.research.google.com/github/vinutha18-m/Yardstick/blob/main/Task1_%26_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
pip install groq

In [86]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technology-driven world, and their importance can be seen in several areas. Here are some reasons why fast language models are essential:

1. **Efficient Processing**: Fast language models can process vast amounts of text data quickly, which is vital for applications that require real-time or near-real-time processing, such as chatbots, virtual assistants, and language translation software.
2. **Improved User Experience**: Quick response times are essential for a seamless user experience. Fast language models enable applications to respond promptly to user queries, reducing wait times and improving overall satisfaction.
3. **Scalability**: As the amount of text data grows, fast language models can handle the increased volume without compromising performance. This scalability is critical for large-scale applications, such as language translation platforms, text summarization tools, and sentiment analysis software.
4. **Real-time Applications**

In [87]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


In [88]:
import os
import json
from typing import List, Dict
import requests

In [78]:
from textwrap import shorten

class ConversationManager:
    def __init__(self, summary_interval=3, char_limit=300, word_limit=50):
        self.history = []
        self.summary_interval = summary_interval
        self.char_limit = char_limit
        self.word_limit = word_limit
        self.assistant_turn_count = 0

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})

        if role == "assistant":
            self.assistant_turn_count += 1

        if self.assistant_turn_count > 0 and self.assistant_turn_count % self.summary_interval == 0:
            self.summarize_history()

    def summarize_history(self):
        # Keep all messages, including system summary
        non_system_entries = [entry for entry in self.history if entry['role'] != 'system']

        if len(non_system_entries) >= 2:
            old_messages = non_system_entries[:-1]  # all except last one
            last_message = non_system_entries[-1:]  # last message

            # Check for existing system summary
            system_summary_index = -1
            for i, entry in enumerate(self.history):
                if entry['role'] == 'system':
                    system_summary_index = i
                    break

            if system_summary_index != -1:
                # Update existing system summary
                old_messages_text = " ".join([self.history[system_summary_index]['content']] + [f"{e['role'].upper()}: {e['content']}" for e in old_messages])
                summary_content = shorten(old_messages_text, width=self.char_limit, placeholder="...")
                self.history[system_summary_index]['content'] = f"{summary_content}"
                self.history = [self.history[system_summary_index]] + last_message
            else:
                # Add new system summary
                old_messages_text = " ".join(f"{e['role'].upper()}: {e['content']}" for e in old_messages)
                summary_content = shorten(old_messages_text, width=self.char_limit, placeholder="...")
                self.history = [{"role": "system", "content": f"{summary_content}"}] + last_message



    def truncate_last_n(self, n):
        self.history = self.history[-n:]

    def truncate_by_char_limit(self, limit):
        total_chars = 0
        truncated = []
        for entry in reversed(self.history):
            entry_length = len(entry['content'])
            if total_chars + entry_length <= limit:
                truncated.insert(0, entry)
                total_chars += entry_length
            else:
                break
        self.history = truncated

    def truncate_by_word_limit(self, limit):
        total_words = 0
        truncated = []
        for entry in reversed(self.history):
            entry_words = len(entry['content'].split())
            if total_words + entry_words <= limit:
                truncated.insert(0, entry)
                total_words += entry_words
            else:
                break
        self.history = truncated

    def get_history(self):
        return self.history

In [80]:
conv_manager = ConversationManager(summary_interval=3, char_limit=300, word_limit=50)

conv_manager.add_message("user", "Hello!")
conv_manager.add_message("assistant", "Hi there, how can I help?")
conv_manager.add_message("user", "Can you explain AI?")  # Summarization happens here

conv_manager.add_message("assistant", "Sure, AI stands for artificial intelligence.")
conv_manager.add_message("user", "Give example")
conv_manager.add_message("assistant", "Like self-driving cars, chatbots, and recommendation systems.")  # Another summary

conv_manager.add_message("user", "What is ML?")
conv_manager.add_message("assistant", "Machine learning is part of AI where machines learn from data.")

# Display final history
for entry in conv_manager.get_history():
    if entry['role'] == "system":
        print(f"SYSTEM: Summary of previous conversation: {entry['content']}")
    else:
        print(f"{entry['role'].upper()}: {entry['content']}")



SYSTEM: Summary of previous conversation: USER: Hello! ASSISTANT: Hi there, how can I help? USER: Can you explain AI? ASSISTANT: Sure, AI stands for artificial intelligence. USER: Give example ASSISTANT: Like self-driving cars, chatbots, and recommendation systems.
USER: What is ML?
ASSISTANT: Machine learning is part of AI where machines learn from data.


**Task 2**

In [91]:
# ================================
# Step 1: Install dependencies
# ================================
!pip install groq jsonschema

# ================================
# Step 2: Imports
# ================================
import os
import json
from groq import Groq
import jsonschema
from jsonschema import validate

# ================================
# Step 3: Set API key
# ================================
# In Colab, you can store securely:
# from google.colab import userdata
# os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

API_KEY = os.environ.get("GROQ_API_KEY")  # make sure key is set in environment
client = Groq(api_key=API_KEY)

# ================================
# Step 4: JSON Schema Definition
# ================================
schema = {
    "name": "extract_user_info",
    "description": "Extract personal details from chat conversations",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string", "format": "email"},
            "phone": {"type": "string", "pattern": "^[+0-9- ]{7,15}$"},
            "location": {"type": "string"},
            "age": {"type": "integer", "minimum": 0, "maximum": 120}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}

# ================================
# Step 5: Extraction Function
# ================================
def extract_info_from_chat(chat_text):
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  # ✅ Groq supported model
        messages=[{"role": "user", "content": chat_text}],
        functions=[schema],
        function_call={"name": "extract_user_info"}
    )

    # ✅ dot-access instead of dict-style
    arguments = response.choices[0].message.function_call.arguments
    return json.loads(arguments)

# ================================
# Step 6: Sample Chats
# ================================
sample_chats = [
    "Hi, my name is Ramesh Kumar. You can reach me at ramesh.kumar@gmail.com. "
    "I live in Bangalore. My phone is +91-9876543210 and I am 29 years old.",

    "This is John. My email is john123@yahoo.com. I’m 35 years old. "
    "My number is 555-123-4567. I live in New York.",

    "My name is Priya Sharma, I’m 22. You can call me at +44 7700 900123. "
    "I stay in London. My email is priya.sharma22@hotmail.com."
]

# ================================
# Step 7: Run + Validate
# ================================
for i, chat in enumerate(sample_chats, 1):
    print(f"\n📌 Chat {i}: {chat}\n")
    extracted = extract_info_from_chat(chat)

    try:
        validate(instance=extracted, schema=schema["parameters"])
        print("✅ Valid JSON extracted:")
        print(json.dumps(extracted, indent=2))
    except jsonschema.exceptions.ValidationError as e:
        print("❌ Validation error:", e.message)



📌 Chat 1: Hi, my name is Ramesh Kumar. You can reach me at ramesh.kumar@gmail.com. I live in Bangalore. My phone is +91-9876543210 and I am 29 years old.

✅ Valid JSON extracted:
{
  "age": 29,
  "email": "ramesh.kumar@gmail.com",
  "location": "Bangalore",
  "name": "Ramesh Kumar",
  "phone": "+91-9876543210"
}

📌 Chat 2: This is John. My email is john123@yahoo.com. I’m 35 years old. My number is 555-123-4567. I live in New York.

✅ Valid JSON extracted:
{
  "age": 35,
  "email": "john123@yahoo.com",
  "location": "New York",
  "name": "John",
  "phone": "555-123-4567"
}

📌 Chat 3: My name is Priya Sharma, I’m 22. You can call me at +44 7700 900123. I stay in London. My email is priya.sharma22@hotmail.com.

✅ Valid JSON extracted:
{
  "age": 22,
  "email": "priya.sharma22@hotmail.com",
  "location": "London",
  "name": "Priya Sharma",
  "phone": "+44 7700 900123"
}
